In [1]:
# do basic imports and unpack McMurdo data

from pmagpy import ipmag
reload(ipmag)
from pmagpy import pmag
from programs import new_builder as nb
from programs import data_model3
reload(data_model3)
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from programs.new_builder import Contribution

import pmagpy.controlled_vocabularies3 as cv



/Users/nebula/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


got full_df
got parsed_df
got full_df
got parsed_df


In [2]:
dir_path = os.path.join(os.getcwd(), '3_0', 'Megiddo')
con = Contribution(dir_path)

loc_dm = con.tables['locations'].data_model.dm['locations']
loc_df = con.tables['locations'].df
site_dm = con.tables['sites'].data_model.dm['sites']
site_df = con.tables['sites'].df


current_con = con
current_df = site_df
current_dm = site_dm
# replace NaN with None
current_dm = current_dm.where((pd.notnull(current_dm)), None)
cols = current_df.columns


#vocab.get_controlled_vocabularies()

# mess up some validations for locations
#current_df.loc['Tel Hazor']['lat_s'] = 400.
#current_df['dir_inc'] = 5
#current_con.tables.pop('sites')

# mess up some validations for sites
current_df.pop('age')
current_df['dir_tilt_correction'] = 1
current_df['dir_tilt_correction'] = 'one'

current_df.head()

-W- No such file: /Users/nebula/Python/PmagPy/3_0/Megiddo/images.txt


,age_high,age_low,age_unit,citations,description,geologic_classes,geologic_types,lat,lithologies,location,lon,site,dir_tilt_correction
site,,,,,,,,,,,,,
hz05,-732,-800,Years Cal AD (+/-),This study,Tel Hazor startum 5,Archeologic,Mixed Archeological Objects,33.017,Not Specified,Tel Hazor,35.568,hz05,one
hz06,-732,-800,Years Cal AD (+/-),This study,Tel Hazor startum 6,Archeologic,Mixed Archeological Objects,33.017,Not Specified,Tel Hazor,35.568,hz06,one
hz07,-800,-900,Years Cal AD (+/-),This study,Tel Hazor startum 7,Archeologic,Mixed Archeological Objects,33.017,Not Specified,Tel Hazor,35.568,hz07,one
hz09,-900,-1000,Years Cal AD (+/-),This study,Tel Hazor startum 9,Archeologic,Mixed Archeological Objects,33.017,Not Specified,Tel Hazor,35.568,hz09,one
hz10,-900,-1000,Years Cal AD (+/-),This study,Tel Hazor startum 10,Archeologic,Mixed Archeological Objects,33.017,Not Specified,Tel Hazor,35.568,hz10,one


In [3]:
import pmagpy.controlled_vocabularies3 as cv
reload(cv)
vocab = cv.Vocabulary()
vocabulary, possible_vocabulary = vocab.get_controlled_vocabularies()



-I- Importing controlled vocabularies from https://earthref.org


In [4]:
# check column validity
invalid_cols = [col for col in cols if col not in current_dm.index]


def requiredUnless(col_name, df, arg, *args):
    arg_list = arg.split(",")
    arg_list = [arg.strip('"') for arg in arg_list]
    msg = ""
    for a in arg_list:
        if "." in a:
            continue
        if a not in df.columns:
            msg += "{} is required unless {} is present.  ".format(col_name, a)
    if msg:
        return msg
    else:
        return None
    return None


def requiredUnlessTable(col_name, df, arg, *args):
    """
    Col_name must be present in df unless
    arg (table_name) is present in contribution
    """
    table_name = arg
    if col_name in df.columns:
        return None
    elif table_name in current_con.tables:
        return None
    else:
        #print "{} is required unless table {} is present".format(col_name, table_name)
        return "{} is required unless table {} is present".format(col_name, table_name)

    
def requiredIfGroup(col_name, df, arg, *args):
    """
    Col_name is required if other columns of 
    the group arg are present.
    """
    group_name = arg
    groups = set()
    columns = df.columns
    for col in columns:
        if col not in current_dm.index:
            continue
        group = current_dm.loc[col]['group']
        groups.add(group)
    if group_name in groups:
        if col_name in columns:
            return None
        else:
            #print "{} is required if column group {} is used".format(col_name, group_name)
            return "{} is required if column group {} is used".format(col_name, group_name)
    return None


def required(col_name, df, arg):
    #print df
    if col_name in df.columns:
        return None
    else:
        return "{} is required".format(col_name)
    

def check_max(row, col_name, arg, *args):
    #print 'check_max'
    cell_value = row[col_name]
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    arg = float(arg)
    try:
        if float(cell_value) <= float(arg):
            return None
        else:
            return "{} must be <= {}".format(str(cell_value), str(arg))
    # this happens when the value isn't a float (an error which will be caught elsewhere)
    except ValueError:
        return None

def check_min(row, col_name, arg, *args):
    #print 'check_min'
    cell_value = row[col_name]
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
        
    try:
        if float(cell_value) >= float(arg):
            return None
        else:
            return "{} must be >= {}".format(str(cell_value), str(arg))
    # this happens when the value isn't a float (an error which will be caught elsewhere)
    except ValueError:
        return None

def cv(row, col_name, arg, current_data_model):
    cell_value = row[col_name]
    if cell_value.lower() in vocabulary[col_name]:
        return None
    else:
        return "{} is not in {}".format(cell_value, arg)
        

# validate presence
presence_operations = {"required": required, "requiredUnless": requiredUnless,
                       "requiredIfGroup": requiredIfGroup, 
                       'requiredUnlessTable': requiredUnlessTable}
# validate values
value_operations = {"max": check_max, "min": check_min, "cv": cv}

def split_func(string):
    """
    Take a string like 'requiredIf("arg_name")'
    return the function name and the argument:
    (requiredIf, arg_name)
    """
    ind = string.index("(")
    return string[:ind], string[ind+1:-1].strip('"')


def test_type(value, value_type):
    if value_type == "String":
        if str(value) == value:
            return None
        else:
            return "should be string"
    elif value_type == "Number":
        try:
            float(value)
            return None
        except ValueError:
            return "should be a number"
    elif value_type == "Integer":
        if int(value) == value:
            return None
        else:
            return "should be an integer"
    else:
        return None
    #String
    # Number
    #Integer
    #List
    #Matrix
    #Dictionary
    

#current_df["presence_pass_age_requiredUnless"] = ''
#print current_df["presence_pass_age_requiredUnless"]
for validation_name, validation in current_dm.iterrows():
    #if validation_name == "age":
    #    continue
    
    # test types first
    #array([u'Number', u'String', u'List', u'Matrix', u'Integer', u'Dictionary'], dtype=object)
    value_type = validation['type']
    #print 'value_type', value_type
    if validation_name in current_df.columns:
        output = current_df[validation_name].apply(test_type, args=(value_type,))
        current_df["type_" + validation_name] = output
    
    
    val_list = validation['validations']
    if not val_list:
        continue
    for num, val in enumerate(val_list):
        #if num > 0:
        #    continue
        # split val into function & arg
        func_name, arg = split_func(val)
        # first validate for presence
        if func_name in presence_operations:
            func = presence_operations[func_name]
            grade = func(validation_name, current_df, arg)
            pass_col_name = "presence_pass_" + validation_name + "_" + func.__name__
            current_df[pass_col_name] = grade
            #print current_df[pass_col_name]

        # then validate for correct values
        elif func_name in value_operations:
            func = value_operations[func_name]
            if validation_name in current_df.columns:
                #print val, "!!"
                #grade = current_df[validation_name].apply(func, args=(validation_name, current_df, arg))
                grade = current_df.apply(func, args=(validation_name, arg, current_dm), axis=1)
                current_df["pass_" + validation_name + "_" + str(num)] = grade.astype(object)
                

# need to test value types....                
                
                
#print current_df["presence_pass_age_requiredUnless"]

#current_df["presence_pass_" + validation_name] = 'hi'
#print current_df["presence_pass_" + validation_name]

  # check that values pass validation
# required()
# requiredUnless
# min
# max
# cv (controlled vocab)
# in
# requiredIfGroup
# type
# sv (suggested vocab)



# re-do upload_magic to use contribution-level (??)

# first, do validations on each table in the contribution
# this will include removing unneded data (RmKeys from old upload_magic)
# this will also include checking everything against the data model (strings are strings, etc.)g

# next, do cross-contribution validations (all specimens exist, etc.)

# next, splat out each table into a file and wrap it up.  give it a sensible name.  

pass_cols = current_df.columns.str.match("^pass_")
present_cols = current_df.columns.str.match("^presen")
type_cols = current_df.columns.str.match("^type_")
val_cols = np.where([bool(v) for v in pass_cols], [bool(v) for v in pass_cols], [bool(v) for v in present_cols])

present_col_names = current_df.columns[present_cols]
current_df[present_col_names].dropna(how='all', axis=1).head()

current_df[current_df.columns[type_cols]].dropna(how='all', axis=1)



,type_dir_tilt_correction
site,
hz05,should be a number
hz06,should be a number
hz07,should be a number
hz09,should be a number
hz10,should be a number
hz11,should be a number
hz13,should be a number
hz15,should be a number
hz18,should be a number
